In [8]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

matplotlib.rcParams.update({
    
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
    'font.size' : 14
})

%config InlineBackend.figure_formats = ['svg']

In [9]:
df = pd.read_csv("log/describe_all_cpu_ruche_complete.csv")
# df = pd.read_csv("log/describe_all_cpu_ruche.csv")
# df_gpu = pd.read_csv("log/describe_all_gpu_ruche_large.csv")
df_gpu = pd.read_csv("log/describe_all_gpu_noBR1D.csv")

# display(df_gpu)


#we drop the false values
df_gpu = df_gpu.drop(df_gpu[(df_gpu['kernel']!="BasicRange1D") & (df_gpu['error_mean'] > 10e-4)].index)

In [10]:
kernels_name = df["kernel"].unique()

values_cpu = {
    #'kernel_name': ([size1,size2,size3], [value1, value2, value3])
}

values_gpu = {}

for kernel_type in kernels_name:
    #fill gpu
    one_kernel_gpu = df_gpu[df_gpu["kernel"] == kernel_type]
    values_gpu[kernel_type] = (one_kernel_gpu["global_size"], one_kernel_gpu['cellspersec_mean'])

    #fill cpu
    one_kernel_cpu = df[df["kernel"] == kernel_type]
    values_cpu[kernel_type] = (one_kernel_cpu["global_size"], one_kernel_cpu['cellspersec_mean'])

In [13]:
fig = plt.figure(figsize=(12,4))#, constrained_layout=True)
ax = fig.add_subplot(111)    # The big subplot
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

# Tracer les données
for key, data in values_cpu.items():
    sizes, perf = data

    ax1.plot(sizes, perf*10e9, 'x-', label=key)
    
    #fake plot to set the legend
    ax.plot(0, 0, label=key)
    

for key, data in values_gpu.items():
    sizes, perf = data
    ax2.plot(sizes, perf*10e9, 'x-', label=key)


box = ax.get_position()

ax.legend(loc='lower center', bbox_to_anchor=(box.width/2 + 0.11, box.width/2 + 0.71), fancybox=True, shadow=True, ncol=5)

#set common labels (i.e. the labels of the large subplot)
ax.set_ylabel('Performance (Updated cells per s)')
ax.set_xlabel('Global size ($n_x \\times n_{vx}$)')

#Turn off axis for the large subplot
ax.spines['top'].set_color('none')
ax.spines['bottom'].set_color('none')
ax.spines['left'].set_color('none')
ax.spines['right'].set_color('none')
ax.tick_params(labelcolor='w', top=False, bottom=False, left=False, right=False)

ax1.set_title('CPU performances')
ax2.set_title('GPU performances')
subfigs = [ax1, ax2]
for subfig in subfigs:
    subfig.grid(True)
    subfig.set_xscale('log', base=10)

plt.show()
# fig.savefig("plot.pdf")

<Figure size 1200x400 with 3 Axes>

In [12]:
KERNEL_TYPE = "Hierarchical"
same_kernel = df[df['kernel']==KERNEL_TYPE]

sizes = same_kernel['global_size']

gbs = same_kernel['throughput_mean']
gbs_std = same_kernel['throughput_std']

errors = same_kernel['error_mean']
errors_std = same_kernel['error_std']

df

,global_size,nx,nvx,kernel,error_mean,error_std,runtime_mean,runtime_std,cellspersec_mean,cellspersec_std,throughput_mean,throughput_std,gpu
0,4096,64,64,BasicRange2D,3.523570e-03,0.0,1.212534,0.175799,0.034328,0.004719,0.549241,0.075506,0.0
1,16384,128,128,BasicRange2D,1.952430e-04,0.0,2.060462,0.416563,0.081865,0.014681,1.309846,0.234898,0.0
2,65536,256,256,BasicRange2D,9.115920e-06,0.0,3.817222,0.118277,0.171814,0.005228,2.749034,0.083643,0.0
3,262144,512,512,BasicRange2D,2.507810e-07,0.0,9.525816,0.661959,0.276317,0.020248,4.421070,0.323962,0.0
4,1048576,1024,1024,BasicRange2D,7.845860e-09,0.0,41.003460,2.512470,0.256512,0.015983,4.104192,0.255724,0.0
5,4194304,1024,4096,BasicRange2D,7.845940e-09,0.0,153.752800,10.863181,0.273826,0.018263,4.381226,0.292218,0.0
6,8388608,1024,8192,BasicRange2D,7.845940e-09,0.0,320.545400,7.599696,0.261815,0.006186,4.189046,0.098983,0.0
7,16777216,1024,16384,BasicRange2D,7.845940e-09,0.0,626.158800,3.286152,0.267945,0.001399,4.287116,0.022375,0.0
8,33554432,1024,32768,BasicRange2D,7.845940e-09,0.0,1281.526000,12.569313,0.261852,0.002569,4.189636,0.041098,0.0
9,67108864,1024,65536,BasicRange2D,7.845940e-09,0.0,2523.150000,24.425187,0.265992,0.002569,4.255872,0.041102,0.0
